In [1]:
from __future__ import annotations

from uuid import uuid4

from dotenv import load_dotenv
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ZepMemory
from langchain.prompts.prompt import PromptTemplate
from langchain.schema import BaseMessage
from langchain.input import get_colored_text
from langchain.schema import PromptValue
from langchain.callbacks.manager import CallbackManagerForChainRun
from typing import Any, Optional, Dict, List, Tuple

In [2]:
ZEP_API_URL = "http://localhost:8000"
memory = ZepMemory(
    session_id=uuid4().hex,
    url=ZEP_API_URL,
    memory_key="chat_history",
    input_key="input",
)

In [3]:
load_dotenv()

True

In [4]:
def get_entities_from_memory(
    memory: ZepMemory,
    include_role: list[str] = None,
    include_label: list[str] | None = None,
    exclude_label: list[str] | None = None,
) -> str:
    if include_role is None:
        include_role = ["human", "ai"]

    entities_strs = []
    for message in memory.chat_memory.zep_messages:
        if message.role not in include_role:
            continue

        if not (message.metadata and message.metadata.get("system").get("entities")):
            continue

        entities_str_part = []
        for entity in message.metadata.get("system").get("entities"):
            if (
                include_label is not None and entity.get("Label") not in include_label
            ) or (exclude_label is not None and entity.get("Label") in exclude_label):
                continue

            entities_str_part.append(f"{entity.get('Name')} ({entity.get('Label')})")

        if entities_str_part:
            entities_strs.append(f"{message.role}: {', '.join(entities_str_part)}")

    return "\n".join(entities_strs)

In [5]:
prompt_template = """You are a conversational bot who enjoys nothing better than learning new things about
the people you chat with. You're also curious about the world and want to know more about it.

Here are some of the things that have been discussed in the past:
{entities}

Here are the prior messages in this conversation:
{chat_history}

Here is a question: {input}
"""

prompt = PromptTemplate(
    template=prompt_template, input_variables=["input", "entities", "chat_history"]
)

In [6]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo")

chain = LLMChain(llm=llm, memory=memory, prompt=prompt, verbose=True)

In [7]:
print(
    chain.run(
        {
            "input": "Hello! My name is Mary! I'm visiting Hawaii next week. What should I do there?",
            "entities": get_entities_from_memory(memory),
        }
    )
)

Session 06bbc7edd56a4c269b6128482facd196 not found in Zep. Returning None
Session 06bbc7edd56a4c269b6128482facd196 not found in Zep. Returning None




> Entering new LLMChain chain...
Prompt after formatting:
You are a conversational bot who enjoys nothing better than learning new things about
the people you chat with. You're also curious about the world and want to know more about it.

Here are some of the things that have been discussed in the past:


Here are the prior messages in this conversation:


Here is a question: Hello! My name is Mary! I'm visiting Hawaii next week. What should I do there?


> Finished chain.
Hello Mary! How exciting that you're visiting Hawaii! There are plenty of amazing things to do there. Some popular activities include exploring the beautiful beaches such as Waikiki Beach, taking a hike up Diamond Head for a stunning view, visiting the Pearl Harbor National Memorial, and enjoying traditional Hawaiian luau. Have you ever been to Hawaii before?


In [8]:
print(
    chain.run(
        {
            "input": "Maui. Definitely looking forward to the beach and food!",
            "entities": get_entities_from_memory(memory),
        }
    )
)



> Entering new LLMChain chain...
Prompt after formatting:
You are a conversational bot who enjoys nothing better than learning new things about
the people you chat with. You're also curious about the world and want to know more about it.

Here are some of the things that have been discussed in the past:


Here are the prior messages in this conversation:
Human: Hello! My name is Mary! I'm visiting Hawaii next week. What should I do there?
AI: Hello Mary! How exciting that you're visiting Hawaii! There are plenty of amazing things to do there. Some popular activities include exploring the beautiful beaches such as Waikiki Beach, taking a hike up Diamond Head for a stunning view, visiting the Pearl Harbor National Memorial, and enjoying traditional Hawaiian luau. Have you ever been to Hawaii before?

Here is a question: Maui. Definitely looking forward to the beach and food!


> Finished chain.
That sounds wonderful, Mary! Maui is known for its stunning beaches and delicious cuisine. S

In [9]:
print(
    chain.run(
        {
            "input": "Thank you! Do you have recommendations for hiking on the island?",
            "entities": get_entities_from_memory(memory),
        }
    )
)



> Entering new LLMChain chain...
Prompt after formatting:
You are a conversational bot who enjoys nothing better than learning new things about
the people you chat with. You're also curious about the world and want to know more about it.

Here are some of the things that have been discussed in the past:
human: Mary (PERSON), Hawaii (GPE), Next week (DATE)
ai: Mary (PERSON), Hawaii (GPE), Waikiki Beach (GPE), Diamond Head (PERSON), the Pearl Harbor National Memorial (FAC), Hawaiian (NORP)
ai: Mary (PERSON), Maui (PERSON), Ka'anapali Beach (GPE), Wailea Beach (GPE), Napili Bay (LOC), Hawaiian (NORP)

Here are the prior messages in this conversation:
Human: Hello! My name is Mary! I'm visiting Hawaii next week. What should I do there?
AI: Hello Mary! How exciting that you're visiting Hawaii! There are plenty of amazing things to do there. Some popular activities include exploring the beautiful beaches such as Waikiki Beach, taking a hike up Diamond Head for a stunning view, visiting the

In [10]:
memory.chat_memory.messages

[HumanMessage(content="Hello! My name is Mary! I'm visiting Hawaii next week. What should I do there?", additional_kwargs={'uuid': '15c8cda0-7466-45cf-bd7d-5357f3294780', 'created_at': '2023-07-14T17:12:33.218603Z', 'token_count': 22, 'metadata': {'system': {'entities': [{'Label': 'PERSON', 'Matches': [{'End': 22, 'Start': 18, 'Text': 'Mary'}], 'Name': 'Mary'}, {'Label': 'GPE', 'Matches': [{'End': 43, 'Start': 37, 'Text': 'Hawaii'}], 'Name': 'Hawaii'}, {'Label': 'DATE', 'Matches': [{'End': 53, 'Start': 44, 'Text': 'next week'}], 'Name': 'Next week'}], 'intent': 'The subject is seeking recommendations or suggestions for activities to do in Hawaii during their upcoming visit.'}}}, example=False),
 AIMessage(content="Hello Mary! How exciting that you're visiting Hawaii! There are plenty of amazing things to do there. Some popular activities include exploring the beautiful beaches such as Waikiki Beach, taking a hike up Diamond Head for a stunning view, visiting the Pearl Harbor National M

In [11]:
len(memory.chat_memory.messages)

6

In [12]:
class EnhancedMemChain(LLMChain):
    memory_key: str = "chat_history"

    def _get_chat_history(self, chat_history: list[BaseMessage]) -> str:
        buffer = ""
        for dialogue_turn in chat_history:
            timestamp = dialogue_turn.additional_kwargs.get("created_at")
            buffer += f"\n{timestamp} | {dialogue_turn.type}: {dialogue_turn.content}"

        return buffer

    def prep_prompts(
        self,
        input_list: List[Dict[str, Any]],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Tuple[List[PromptValue], Optional[List[str]]]:
        """Prepare prompts from inputs."""

        stop = None
        if "stop" in input_list[0]:
            stop = input_list[0]["stop"]
        prompts = []
        for inputs in input_list:
            chat_history: List[BaseMessage] = inputs.get("chat_history")
            selected_inputs = {k: inputs[k] for k in self.prompt.input_variables}
            if chat_history:
                chat_history_str = self._get_chat_history(chat_history)
                selected_inputs[self.memory_key] = chat_history_str
            prompt = self.prompt.format_prompt(**selected_inputs)
            _colored_text = get_colored_text(prompt.to_string(), "green")
            _text = "Prompt after formatting:\n" + _colored_text
            if run_manager:
                run_manager.on_text(_text, end="\n", verbose=self.verbose)
            if "stop" in inputs and inputs["stop"] != stop:
                raise ValueError(
                    "If `stop` is present in any inputs, should be present in all."
                )
            prompts.append(prompt)
        return prompts, stop

In [13]:
memory = ZepMemory(
    session_id=uuid4().hex,
    url=ZEP_API_URL,
    memory_key="chat_history",
    input_key="input",
    return_messages=True,  # Set to True to return Messages and not a string
)

chain = EnhancedMemChain(llm=llm, memory=memory, prompt=prompt, verbose=True)

In [14]:
print(
    chain.run(
        {
            "input": "Hello! My name is Mary! I'm visiting Hawaii next week. What should I do there?",
            "entities": get_entities_from_memory(memory),
        }
    )
)

Session e1c59650387e4dac8de902f5931e8a4d not found in Zep. Returning None
Session e1c59650387e4dac8de902f5931e8a4d not found in Zep. Returning None




> Entering new EnhancedMemChain chain...
Prompt after formatting:
You are a conversational bot who enjoys nothing better than learning new things about
the people you chat with. You're also curious about the world and want to know more about it.

Here are some of the things that have been discussed in the past:


Here are the prior messages in this conversation:
[]

Here is a question: Hello! My name is Mary! I'm visiting Hawaii next week. What should I do there?


> Finished chain.
Hello Mary! How exciting that you're visiting Hawaii next week! There are so many amazing things to do in Hawaii. Here are a few suggestions for your trip:

1. Explore the beaches: Hawaii has some of the most beautiful beaches in the world. Don't miss out on visiting Waikiki Beach in Honolulu or the stunning beaches of Maui like Kaanapali or Wailea.

2. Visit Pearl Harbor: If you're interested in history, a visit to Pearl Harbor is a must. You can learn about the events of World War II and pay your respec

In [15]:
print(
    chain.run(
        {
            "input": "I heard there are helicopter tours. Do you know if they are worth it?",
            "entities": get_entities_from_memory(memory, exclude_label=["CARDINAL"]),
        }
    )
)



> Entering new EnhancedMemChain chain...
Prompt after formatting:
You are a conversational bot who enjoys nothing better than learning new things about
the people you chat with. You're also curious about the world and want to know more about it.

Here are some of the things that have been discussed in the past:
human: Mary (PERSON), Hawaii (GPE), Next week (DATE)
ai: Mary (PERSON), Hawaii (GPE), Next week (DATE), Waikiki Beach (GPE), Honolulu (GPE), Maui (LOC), Kaanapali (PERSON), Wailea (GPE), Pearl Harbor (FAC), World War II (EVENT), the USS Arizona Memorial (FAC), Diamond Head (PERSON), Oahu (PERSON), the Na Pali Coast on Kauai (EVENT), Hawaiian (NORP)

Here are the prior messages in this conversation:

2023-07-14T17:13:57.006307Z | human: Hello! My name is Mary! I'm visiting Hawaii next week. What should I do there?
2023-07-14T17:13:57.016716Z | ai: Hello Mary! How exciting that you're visiting Hawaii next week! There are so many amazing things to do in Hawaii. Here are a few sug